# Memory

In [1]:
!pip install --upgrade --quiet langchain langchain-community langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.1/606.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00


In [2]:
import os


In [3]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [4]:
import os
os.environ['OPENAI_API_KEY']=OPENAI_API_KEY

In [5]:
from langchain_openai import ChatOpenAI

In [6]:
llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

In [7]:
from langchain_core.messages import AIMessage, HumanMessage

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [9]:
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "you are a helpful assistant. Answer all the question to the best of your ability."
    ),
    MessagesPlaceholder(variable_name = "messages"),
]
)

In [10]:
chain =  prompt | llm

In [13]:
resposne = chain.invoke(
    {
       "messages": [

                   HumanMessage(
                       content= "translate this given sentence from english to french: I LOVE AI."
                   ),
                   AIMessage(content= "J'adore la AI."),

                   HumanMessage(content= "what did you just say?"),
       ] ,

    }
)

In [14]:
resposne.content

'I said "I love AI" in French.'

In [15]:
from langchain.memory import ChatMessageHistory

In [16]:
demo_chat_history=ChatMessageHistory()

In [17]:
demo_chat_history.add_user_message("translate this given sentence from english to french: I LOVE AI.")

In [18]:
demo_chat_history.add_ai_message("J'adore la AI.")

In [19]:
demo_chat_history.messages

[HumanMessage(content='translate this given sentence from english to french: I LOVE AI.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="J'adore la AI.", additional_kwargs={}, response_metadata={})]

In [20]:
demo_chat_history=ChatMessageHistory()

In [21]:
input1="who is a prime minister of india?"

In [22]:
demo_chat_history.add_user_message(input1)

In [23]:
demo_chat_history.messages

[HumanMessage(content='who is a prime minister of india?', additional_kwargs={}, response_metadata={})]

In [24]:
resposne=chain.invoke(
    {
        "messages": demo_chat_history.messages
    }
)



In [25]:
resposne.content

'As of October 2021, the Prime Minister of India is Narendra Modi. He has been in office since May 2014 after his party, the Bharatiya Janata Party (BJP), won the general elections.'

In [26]:
demo_chat_history.add_ai_message(resposne)

In [27]:
input2="what did i ask to you just now?"

In [28]:
demo_chat_history.add_user_message(input2)

In [29]:
demo_chat_history.messages

[HumanMessage(content='who is a prime minister of india?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='As of October 2021, the Prime Minister of India is Narendra Modi. He has been in office since May 2014 after his party, the Bharatiya Janata Party (BJP), won the general elections.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 36, 'total_tokens': 83, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_c66b5540ac', 'id': 'chatcmpl-BExJPadtYggQZMuUqlyzU2eq5CksV', 'finish_reason': 'stop', 'logprobs': None}, id='run-819996d0-24c2-4617-b8f6-0f8b15777350-0', usage_metadata={'input_tokens': 36, 'output_tokens': 47, 'total_tokens': 83, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'o

In [30]:
response2 = chain.invoke(
    {
        "messages": demo_chat_history.messages
    }
)

In [31]:
response2.content

"You asked about the Prime Minister of India, and I provided the current Prime Minister's name."

In [32]:
prompt2 = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain2 = prompt | llm

In [33]:
prompt3 = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

chain = prompt3 | llm


In [34]:
from langchain_core.runnables.history import RunnableWithMessageHistory

In [35]:
demo_chat_history2 = ChatMessageHistory()

In [36]:
chain_with_message_history = RunnableWithMessageHistory(
                              chain,
                              lambda session_id: demo_chat_history2,
                              input_message_key = "input",
                              history_message_key ="chat_history",
                              )

In [37]:
chain_with_message_history.invoke(
    {"input": "Translate this sentence from English to French: I love programming."},
    {"configurable": {"session_id": "unused"}}
)

AIMessage(content='The translation of "I love programming" from English to French is "J\'adore programmer."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 52, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_c66b5540ac', 'id': 'chatcmpl-BExK5A5m0NOEVep1syPZWgxF7RXuB', 'finish_reason': 'stop', 'logprobs': None}, id='run-8abed242-7a52-4abf-a2a2-4f5970ea43d9-0', usage_metadata={'input_tokens': 52, 'output_tokens': 21, 'total_tokens': 73, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [38]:
chain_with_message_history.invoke(
    {"input": "What did I just ask you?"}, {"configurable": {"session_id": "unused"}}
)

AIMessage(content='You asked me to translate the sentence "I love programming" from English to French, and I provided the translation as "J\'adore programmer."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 351, 'total_tokens': 382, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_c66b5540ac', 'id': 'chatcmpl-BExKBkvcaEIQHmOstl1APtXhCAN2a', 'finish_reason': 'stop', 'logprobs': None}, id='run-63d007ec-e2f7-4f6b-b68d-faa52528f6be-0', usage_metadata={'input_tokens': 351, 'output_tokens': 31, 'total_tokens': 382, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [39]:
from langchain import OpenAI

In [40]:
'''llm = OpenAI(
	temperature=0,
 model_name="gpt-3.5-turbo-1106"
)
 '''

'llm = OpenAI(\n\ttemperature=0,\n model_name="gpt-3.5-turbo-1106"\n)\n '

In [41]:
from langchain.chains import ConversationChain

In [42]:
conversation = ConversationChain(llm=llm)

<ipython-input-42-07a8be19d9f8>:1: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(llm=llm)
/usr/local/lib/python3.11/dist-packages/pydantic/main.py:214: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [43]:
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [44]:
question1="i am interested in exploring the potential of large language model with external knowledge"
question2="i would like to analysis all the different possibility, what can you think of?"
question3="which data source tyoe could be used to give context to the model?"

In [45]:
from langchain.chains.conversation.memory import ConversationBufferMemory

In [46]:
conversation_buf = ConversationChain(
    llm=llm,
    memory= ConversationBufferMemory()

)

In [47]:
conversation_buf("good morning my AI")

<ipython-input-47-7bcbf6b1846f>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  conversation_buf("good morning my AI")


{'input': 'good morning my AI',
 'history': '',
 'response': 'Good morning! How are you today?'}

In [48]:
from langchain.callbacks import get_openai_callback

In [49]:
def count_tokens(chain,query):
  with get_openai_callback() as cb:
    result= chain.run(query)
    print(f"totla no of token is {cb.total_tokens}")
  return result

In [50]:
question1="i am interested in exploring the potential of large language model with external knowledge"

In [51]:
count_tokens(conversation_buf,question1)

<ipython-input-49-f921a4badc7d>:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result= chain.run(query)


totla no of token is 197


"That's a fascinating topic! Large language models like GPT-3 are trained on vast amounts of text data, which allows them to generate human-like responses and even make inferences based on external knowledge sources. There are also efforts to integrate external knowledge bases like Wikipedia or scientific literature into these models to enhance their understanding and reasoning capabilities. It's an exciting area of research with a lot of potential for applications in natural language understanding and generation. Is there something specific you'd like to know about this topic?"

In [52]:
conversation_buf(question1)

{'input': 'i am interested in exploring the potential of large language model with external knowledge',
 'history': "Human: good morning my AI\nAI: Good morning! How are you today?\nHuman: i am interested in exploring the potential of large language model with external knowledge\nAI: That's a fascinating topic! Large language models like GPT-3 are trained on vast amounts of text data, which allows them to generate human-like responses and even make inferences based on external knowledge sources. There are also efforts to integrate external knowledge bases like Wikipedia or scientific literature into these models to enhance their understanding and reasoning capabilities. It's an exciting area of research with a lot of potential for applications in natural language understanding and generation. Is there something specific you'd like to know about this topic?",
 'response': "As I mentioned earlier, integrating external knowledge sources into large language models can enhance their underst

In [53]:
question2="i would like to analysis all the different possibility, what can you think of?"

In [54]:
conversation_buf(question2)

{'input': 'i would like to analysis all the different possibility, what can you think of?',
 'history': "Human: good morning my AI\nAI: Good morning! How are you today?\nHuman: i am interested in exploring the potential of large language model with external knowledge\nAI: That's a fascinating topic! Large language models like GPT-3 are trained on vast amounts of text data, which allows them to generate human-like responses and even make inferences based on external knowledge sources. There are also efforts to integrate external knowledge bases like Wikipedia or scientific literature into these models to enhance their understanding and reasoning capabilities. It's an exciting area of research with a lot of potential for applications in natural language understanding and generation. Is there something specific you'd like to know about this topic?\nHuman: i am interested in exploring the potential of large language model with external knowledge\nAI: As I mentioned earlier, integrating ext

In [55]:
count_tokens(conversation_buf,question2)

totla no of token is 714


'Some potential applications of large language models with external knowledge include:\n\n1. Question answering: These models can provide more accurate and informative answers by leveraging knowledge sources like Wikipedia or other databases.\n\n2. Language translation: By incorporating external knowledge, these models can improve the accuracy and contextuality of translated text.\n\n3. Summarization: Large language models can use external knowledge to generate more informative and concise summaries of text documents.\n\n4. Dialogue generation: By integrating external knowledge, these models can have more meaningful and contextually relevant conversations.\n\n5. Information retrieval: These models can benefit from external knowledge sources to better understand and retrieve relevant information.\n\n6. Personalized content generation: Utilizing external knowledge can help these models create personalized and context-aware content for users.\n\nThese are just a few examples, and there ar

Some potential areas of exploration could include the impact of different types of external knowledge sources on the performance of large language models, the development of more efficient algorithms for integrating external knowledge, and the ethical considerations of using external knowledge in language generation. Additionally, you could also explore the potential applications of large language models with external knowledge in various industries such as healthcare, finance, and customer service. There are also opportunities to investigate the potential limitations and challenges of these models, such as biases in the external knowledge sources and the potential for misinformation. These are just a few possibilities, and there are many more aspects to consider in exploring the potential of large language models with external knowledge.

In [56]:
question3

'which data source tyoe could be used to give context to the model?'

In [57]:
conversation_buf(question3)['response']

'External knowledge sources such as open data repositories, scientific literature, encyclopedias, databases, and domain-specific knowledge bases can be used to provide context to large language models. These sources can contain valuable information and facts that can help the model understand and generate more accurate and contextually relevant responses. Additionally, leveraging structured data sources, such as knowledge graphs or ontologies, can also provide a rich source of context for the model to enhance its understanding and reasoning capabilities. By integrating these diverse data sources, large language models can improve their ability to generate human-like responses and make inferences based on external knowledge.'

# take atleast 5 data source(1 ppt, 1 weblodaer, 1 pdf, 1txt, 1csv)

# maintain one knowledge base(mongodb, astradb , pineconde, weviate)

# user will ask you have to provide answer based on the asked question using this know;ledge base

# handle the comman question as well like hi hello how are you good evening good morning etc.

# you have to mention the complete memory of the conversation here the threshold is 10

# then create a UI for you bot

https://www.youtube.com/@sunnysavita10/videos

